In [ ]:
import pandas as pd

# Assuming the file is uploaded to the /content/ directory
data_path = '/content/df_OHLCV_stocks_etfs.parquet'

df_ohlcv = pd.read_parquet(data_path, engine='pyarrow')

# Display the first few rows to confirm it's loaded correctly
display(df_ohlcv.head())

In [2]:
import pandas as pd

data_path = r'c:\Users\ping\Files_win10\python\py311\stocks\data\df_OHLCV_stocks_etfs.parquet'
df_ohlcv = pd.read_parquet(data_path, engine='pyarrow')
print(f'df_ohlcv.info():\n{df_ohlcv.info()}')
df_ohlcv

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1201017 entries, ('ACN', Timestamp('2001-07-19 00:00:00')) to ('ZS', Timestamp('2025-12-12 00:00:00'))
Data columns (total 5 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   Adj Open   1201017 non-null  float64
 1   Adj High   1201017 non-null  float64
 2   Adj Low    1201017 non-null  float64
 3   Adj Close  1201017 non-null  float64
 4   Volume     1201017 non-null  int64  
dtypes: float64(4), int64(1)
memory usage: 51.0+ MB
df_ohlcv.info():
None


Adj Open  Adj High   Adj Low  Adj Close    Volume
Ticker Date                                                         
ACN    2001-07-19   10.5544   10.6872   10.4845    10.6034  50065558
       2001-07-20   10.5195   10.5195   10.3448    10.4915  13217311
       2001-07-23   10.4845   10.4915   10.1700    10.4845  10731511
       2001-07-24   10.4496   10.4636   10.2749    10.3867   5060735
       2001-07-25   10.2749   10.4496   10.2399    10.4496   6020434
...                     ...       ...       ...        ...       ...
ZS     2025-12-08  243.5000  249.6400  243.4600   244.8800   2077300
       2025-12-09  244.0100  244.7200  241.2900   243.0100   1518200
       2025-12-10  242.2400  245.1200  238.8500   243.2900   1692700
       2025-12-11  241.2000  243.4900  237.8700   242.0800   1294300
       2025-12-12  241.6250  243.0650  233.9380   236.2800   1826614

[1201017 rows x 5 columns]

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ==============================================================================
# 1. IMPROVED DATA PIPELINE (With Z-Score Normalization)
# ==============================================================================

def get_rolling_z_score(data_matrix, window=60):
    """
    Normalizes data so the Agent sees 'Standard Deviations' instead of 'Percent'.
    This solves the Vanishing Gradient problem.
    """
    df = pd.DataFrame(data_matrix)
    # Calculate rolling mean and std
    roll_mean = df.rolling(window=window, min_periods=window).mean()
    roll_std = df.rolling(window=window, min_periods=window).std()

    # Z-Score = (Value - Mean) / StdDev
    z_scores = (df - roll_mean) / (roll_std + 1e-8) # 1e-8 prevents divide by zero

    # Fill NaN (the first 60 days) with 0
    return z_scores.fillna(0.0).values

def prepare_robust_data(df_ohlcv, split_ratio=0.8):
    print("--- 🛠️ Preparing Level 2 Data (Normalized) ---")

    # 1. Pivot
    closes = df_ohlcv['Adj Close'].unstack(level=0)

    # 2. Log Returns
    log_returns = np.log(closes / closes.shift(1)).fillna(0.0)

    # 3. NORMALIZE (The Fix)
    # We pass the NORMALIZED data to the Input (X)
    # But we keep raw log_returns for the Reward Calculation (y)
    print("    > Calculating Rolling Z-Scores (60d)...")
    norm_features = get_rolling_z_score(log_returns.values)

    # 4. Split
    split_idx = int(len(log_returns) * split_ratio)

    # Train Sets
    X_train = norm_features[:split_idx]
    y_train = log_returns.values[:split_idx] # Rewards must be real $

    # Test Sets
    X_test = norm_features[split_idx:]
    y_test = log_returns.values[split_idx:]

    return X_train, y_train, X_test, y_test

# ==============================================================================
# 2. IMPROVED AGENT (Gradient Clipping & Scaling)
# ==============================================================================

# ==============================================================================
# LONG ONLY AGENT (0% to 100% Invested)
# ==============================================================================

class LongOnlyAgent:
    def __init__(self, input_dim, learning_rate=0.001):
        self.weights = np.random.randn(input_dim) * 0.1
        self.lr = learning_rate
        self.velocity = np.zeros_like(self.weights)

    def _sigmoid(self, x):
        # Stable Sigmoid function: Maps any number to range [0.0, 1.0]
        # 0.0 = 100% Cash (Safe)
        # 1.0 = 100% Long (Invested)
        return 1 / (1 + np.exp(-x))

    def predict(self, states):
        # 1. Compute Linear Logits
        logits = np.dot(states, self.weights)

        # 2. Apply Sigmoid (Forces output to be positive)
        return self._sigmoid(logits)

    def train_step(self, states, actions, market_returns):
        # REWARD LOGIC:
        # If Market Drops (-5%) and Action is 0.0 (Cash) -> Reward = 0.0
        # If Market Drops (-5%) and Action is 1.0 (Long) -> Reward = -5.0
        # Result: Agent learns that 0.0 > -5.0, so it learns to go to Cash during crashes.

        # Scale rewards for better gradients
        scaled_rewards = (actions * market_returns) * 100.0

        # 3. DERIVATIVE CHANGE
        # Derivative of Tanh is (1 - x^2)
        # Derivative of Sigmoid is x * (1 - x)
        d_sigmoid = actions * (1 - actions)

        # Standard Gradient Calculation
        grads = states * d_sigmoid[:, None] * scaled_rewards[:, None]
        mean_grad = np.mean(grads, axis=0)

        # Optimizer (Momentum)
        self.velocity = (0.9 * self.velocity) + (0.1 * mean_grad)
        self.weights += self.lr * self.velocity

        return np.mean(actions * market_returns)

class BatchEnv:
    def __init__(self, X, y, lookback=20):
        self.X = X # Normalized Inputs
        self.y = y # Raw Returns (for PnL)
        self.lookback = lookback
        self.n_days, self.n_assets = X.shape

    def get_batch(self, batch_size=2048):
        asset_idx = np.random.randint(0, self.n_assets, size=batch_size)
        day_idx = np.random.randint(0, self.n_days - self.lookback - 1, size=batch_size)

        X_batch = []
        y_batch = []

        for i in range(batch_size):
            t, a = day_idx[i], asset_idx[i]
            X_batch.append(self.X[t : t + self.lookback, a])
            y_batch.append(self.y[t + self.lookback, a])

        return np.array(X_batch), np.array(y_batch)

# ==============================================================================
# 3. RUNNER
# ==============================================================================

def run_level2_experiment(df_ohlcv):
    # A. Data
    X_train, y_train, X_test, y_test = prepare_robust_data(df_ohlcv)

    # B. Setup
    LOOKBACK = 60

    EPOCHS = 1500 # More epochs
    BATCH_SIZE = 4096 # Bigger batch

    env = BatchEnv(X_train, y_train, lookback=LOOKBACK)
    agent = LongOnlyAgent(input_dim=LOOKBACK, learning_rate=0.05)

    history = []

    print("\n🚀 Starting Level 2 Training (Normalized)...")

    for i in range(EPOCHS):
        X, y = env.get_batch(BATCH_SIZE)
        actions = agent.predict(X)
        avg_pnl = agent.train_step(X, actions, y)
        history.append(avg_pnl)

        if i % 200 == 0:
            # Check if weights are moving
            w_norm = np.linalg.norm(agent.weights)
            print(f"Epoch {i:4d} | Batch PnL: {avg_pnl*100:.5f}% | Weight Mag: {w_norm:.4f}")

    # C. Test
    print("\n🧪 Running Validation...")
    test_env = BatchEnv(X_test, y_test, lookback=LOOKBACK)
    X_val, y_val = test_env.get_batch(10000)

    actions_val = agent.predict(X_val)
    results = actions_val * y_val
    final_score = np.mean(results)

    # D. Stats
    win_rate = np.mean(results > 0)
    print(f"FINAL TEST RESULT: Avg PnL: {final_score*100:.5f}%")
    print(f"Win Rate: {win_rate*100:.1f}% (Random is 50%)")
    print(f"LOOKBACK: {LOOKBACK}")
    print(f"LEARNING RATE: {agent.lr}")

# ==============================================================================
    # E. VISUALIZATION (Updated with Negative Lookback Axis)
    # ==============================================================================
    plt.figure(figsize=(12, 5))

    # --- Left Chart: Training Curve ---
    plt.subplot(1, 2, 1)
    plt.plot(pd.Series(history).rolling(100).mean())
    plt.title(f"Training Curve (LR: {agent.lr})")
    plt.ylabel("Avg PnL (Reward)")
    plt.xlabel("Epochs")
    plt.grid(True, alpha=0.3)

    # --- Right Chart: Strategy Weights ---
    plt.subplot(1, 2, 2)

    # Create X-Axis values: from -20 to -1
    # np.arange(-20, 0) generates [-20, -19, ... , -1]
    days_lookback = np.arange(-LOOKBACK, 0)

    # Plot bars using the negative x-values
    plt.bar(days_lookback, agent.weights)

    plt.title("Learned Strategy Weights")
    plt.ylabel("Weight Influence (Negative=Avoid, Positive=Buy)")
    plt.xlabel("Days Lookback (Yesterday = -1)")

    # Force the X-axis to show every integer tick
    plt.xticks(days_lookback, fontsize=8)

    # Add a zero line for clarity
    plt.axhline(0, color='black', lw=1)
    plt.grid(axis='y', alpha=0.3)

    plt.tight_layout()
    plt.show()

# Run
run_level2_experiment(df_ohlcv)

In [5]:
X_train, y_train, X_test, y_test = prepare_robust_data(df_ohlcv)

--- 🛠️ Preparing Level 2 Data (Normalized) ---
    > Calculating Rolling Z-Scores (60d)...


In [8]:
print(f'df_ohlcv.shape: {df_ohlcv.shape}')
X_train.shape

df_ohlcv.shape: (1201017, 5)


(12876, 162)